In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import html5lib
import re
import pandas as pd

In [7]:
def get_soup_by_url(url,verify_boo):
    res = requests.get(url , verify=verify_boo) ### requested url got untrusted SSL certificate, so have to ignore by verify-False
    res.encoding = "utf-8"
    print(res)
    if res.status_code == 200:
        # print("200 Success")
        return (True, BeautifulSoup(res.text, "html.parser"))
    else:
        print(res.status_code)
        return (False, res.status_code)

In [19]:
def get_soup_dict(soup_in):  #soup_in is soup in final url that contains detailed target (aka actual individual news)
    ### Extract body contentx
    soup_in_pbody = soup_in.find_all("div",class_="panel-body")
    cur_pbody = soup_in_pbody[0] 

    ### Extract desired text
    cur_head = cur_pbody.find("div",class_="panel-heading clearfix Circular")
    cur_hidden = cur_pbody.find("p",class_="col-xs-8 remove-xs color7")
    cur_date = cur_pbody.find("div",class_="col-xs-12 col-sm-6 col-md-5 news-2 font_level3 text-right")
    if cur_date is not None:
        clean_cur_date = cur_date.text.strip()[:cur_date.text.strip().find("\n")]
    else:
        clean_cur_date = None
    clean_cur_date
    cur_h3 = cur_pbody.h3
    cur_title = cur_pbody.find("p",class_="font_level2 Circular color3")
    cur_detail = cur_pbody.find("div",class_="col-xs-12 padding-sm1 news-2 Circular")

    ### Create dict
    cur_content = {'panel_heading': cur_head.text.strip(),
               'hidden_date': cur_hidden.text.strip(),
               'date': clean_cur_date,
               'h3': cur_h3.text.strip(),
               'title': cur_title.text.strip(),
               'detail': cur_detail.text.strip()               
               }
    
    return  cur_content

In [20]:
url = "https://www.thaigov.go.th/news/contents/index/1175" ## หน้าข่าวทำเนียบรัฐบาล


In [21]:
### Create df for receiving data
content_news = pd.DataFrame({"panel_heading":[],
                             "url":[],
                             "hidden_date":[],
                             "date":[],
                             "h3":[],
                             "title":[],
                             "detail":[]
                             })

soup = get_soup_by_url(url,False)
error_urls = []

if soup[0] == True:
    soup_pbody = soup[1].find_all("div",class_="panel-body")
    soup_a = soup_pbody[0].find_all("a") ### list all 10 url of news content on each per_page

    ### get soup_dict of each url in page
    for a in soup_a:
        soup_in_url = a.get("href")
        soup_in = get_soup_by_url(soup_in_url,False)
        if soup_in[0] == True:
            cur_dict = get_soup_dict(soup_in[1])
            cur_dict["url"] = soup_in_url
            # print(cur_dict)
            content_news.loc[len(content_news)]=cur_dict
        else:
            error_urls.append(url,soup_in[1])
    
    ### get number of max page for page looping
    soup_page = soup[1].find_all("ul",class_="pagination color2")
    soup_page_li = soup_page[0].find_all("li")
    max_page = int(soup_page_li[-1].a['data-ci-pagination-page'])

    ### Start looping next page
    url_get_page = "https://www.thaigov.go.th/news/contents/index/1175?per_page="

    for i in range(169,max_page+1):
        url_page = url_get_page+str(i*10) ##req per_page is to load 10 contents on each page
        soup = get_soup_by_url(url_page,False)

        if soup[0] == True:
            soup_pbody = soup[1].find_all("div",class_="panel-body")
            soup_a = soup_pbody[0].find_all("a") ### list all 10 url of news content on each per_page

            ### get soup_dict of each url in page
            for a in soup_a:
                soup_in_url = a.get("href")
                soup_in = get_soup_by_url(soup_in_url,False)
                if soup_in[0] == True:
                    cur_dict = get_soup_dict(soup_in[1])
                    cur_dict["url"] = soup_in_url
                    # print(cur_dict)
                    content_news.loc[len(content_news)]=cur_dict
                else:
                    error_urls.append(url,soup_in[1])
        else:
            # print("URL error: "+url)
            error_urls.append([url,soup[1]])

    
else:
    # print("URL error: "+url)
    error_urls.append([url,soup[1]])


/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.thaigov.go.th'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>


/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.thaigov.go.th'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>


/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.thaigov.go.th'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

<Response [200]>


/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.thaigov.go.th'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>


/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.thaigov.go.th'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>


In [22]:
content_news

,panel_heading,url,hidden_date,date,h3,title,detail
0,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันอังคารที่ 18 เมษายน 2566,12/04/2566,เลขาธิการนายกรัฐมนตรี เป็นประธานพิธีทำบุญอาคาร...,เลขาธิการนายกรัฐมนตรี เป็นประธานพิธีทำบุญอาคาร...,วันนี้ (12 เมษายน 2566) เวลา 10.30 น. ณ บริเวณ...
1,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันศุกร์ที่ 18 พฤศจิกายน 2565,18/11/2565,ภริยานายกรัฐมนตรีนำคณะคู่สมรสผู้นำเขตเศรษฐกิจเ...,ภริยานายกรัฐมนตรีนำคณะคู่สมรสผู้นำเขตเศรษฐกิจเ...,วันนี้ (18 พ.ย. 65) เวลา 9.30 น. รศ.นราพร จันท...
2,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันเสาร์ที่ 16 กรกฎาคม 2565,16/07/2565,ชี้แจงประเด็นความต่อเนื่องของการประชุมคณะกรรมก...,ชี้แจงประเด็นความต่อเนื่องของการประชุมคณะกรรมก...,วันที่ 15 กรกฎาคม 2565 นายศุภฤกษ์ ภู่พงศ์ศักดิ...
3,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันพฤหัสบดีที่ 30 มิถุนายน 2565,30/06/2565,ศบค. เผย สธ. ติดตามกรณีเชื้อกลายพันธุ์ BA.4 - ...,ศบค. เผย สธ. ติดตามกรณีเชื้อกลายพันธุ์ BA.4 - ...,วันนี้ (30 มิถุนายน 2565) 12.30 ณ โถงกลาง ตึกส...
4,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันอังคารที่ 14 มิถุนายน 2565,14/06/2565,รัฐบาลเตรียมจัดงานเฉลิมพระเกียรติสมเด็จพระนางเ...,รัฐบาลเตรียมจัดงานเฉลิมพระเกียรติสมเด็จพระนางเ...,วันที่ 14 มิถุนายน 2565 นายธีรภัทร ประยูรสิทธิ...
5,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันอังคารที่ 24 พฤษภาคม 2565,24/05/2565,เลขาธิการนายกรัฐมนตรีเป็นผู้แทนนายกรัฐมนตรี รั...,เลขาธิการนายกรัฐมนตรีเป็นผู้แทนนายกรัฐมนตรี รั...,วันนี้ (24 พฤษภาคม 2565) เวลา 14.30 น. ณ ห้องร...
6,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันจันทร์ที่ 23 พฤษภาคม 2565,23/05/2565,รมว.กก.แถลงข่าวต้อนรับ Amazean Jungle Trail By...,รมว.กก.แถลงข่าวต้อนรับ Amazean Jungle Trail By...,วันที่ 23 พฤษภาคม 2565 เวลา 14.00 น. ณ ศูนย์แถ...
7,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันจันทร์ที่ 23 พฤษภาคม 2565,23/05/2565,รายงานสรุปผลการดำเนินงานที่สำคัญ ของสำนักงานส่...,รายงานสรุปผลการดำเนินงานที่สำคัญของสำนักงานส่ง...,สำนักงานส่งเสริมวิสาหกิจขนาดกลางและขนาดย่อม ได...
8,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันพฤหัสบดีที่ 12 พฤษภาคม 2565,12/05/2565,ผู้ช่วยโฆษก ศบค. ยืนยันการเดินทางในรูปแบบ Thai...,ผู้ช่วยโฆษก ศบค. ยืนยันการเดินทางในรูปแบบ Thai...,วันนี้ (12 พฤษภาคม 2565) เวลา 12.30 น. ณ โถงกล...
9,ข่าวทำเนียบรัฐบาล,https://www.thaigov.go.th/news/contents/detail...,วันพุธที่ 11 พฤษภาคม 2565,11/05/2565,ปลัดสำนักนายกรัฐมนตรี ให้การต้อนรับเด็กและเยาว...,ปลัดสำนักนายกรัฐมนตรี ให้การต้อนรับเด็กและเยาว...,วันที่ 11 พ.ค. 65 เวลา 08.45 น. ณ บริเวณสนามหญ...


In [ ]:
content_news.to_csv("thaigov_news_01.csv")